In [5]:
import wx
from PIL import Image

def xor_crypt(data, key):
    return ''.join(chr(ord(c) ^ ord(k)) for c, k in zip(data, key * len(data)))

def genData(data):
    newd = []
    for i in data:
        newd.append(format(ord(i), '08b'))
    return newd

def modPix(pix, data):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pix = [value for value in imdata.__next__()[:3] +
                                imdata.__next__()[:3] +
                                imdata.__next__()[:3]]
        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j] % 2 != 0):
                pix[j] -= 1
            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if(pix[j] != 0):
                    pix[j] -= 1
                else:
                    pix[j] += 1
        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if(pix[-1] != 0):
                    pix[-1] -= 1
                else:
                    pix[-1] += 1
        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= 1
        pix = tuple(pix)
        yield pix[0:3]
        yield pix[3:6]
        yield pix[6:9]

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)
    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        if (x == w - 1):
            x = 0
            y += 1
        else:
            x += 1

class SteganographyApp(wx.Frame):
    def __init__(self, *args, **kw):
        super(SteganographyApp, self).__init__(*args, **kw)
        self.InitUI()

    def InitUI(self):
        pnl = wx.Panel(self)
        vbox = wx.BoxSizer(wx.VERTICAL)

        self.encodeBtn = wx.Button(pnl, label='Encode')
        self.decodeBtn = wx.Button(pnl, label='Decode')
        self.textCtrl = wx.TextCtrl(pnl, style=wx.TE_MULTILINE)

        vbox.Add(self.encodeBtn, 0, wx.ALL | wx.CENTER, 5)
        vbox.Add(self.decodeBtn, 0, wx.ALL | wx.CENTER, 5)
        vbox.Add(self.textCtrl, 1, wx.EXPAND | wx.ALL, 5)

        self.encodeBtn.Bind(wx.EVT_BUTTON, self.OnEncode)
        self.decodeBtn.Bind(wx.EVT_BUTTON, self.OnDecode)

        pnl.SetSizer(vbox)

        self.SetSize((350, 250))
        self.SetTitle('Steganography App')
        self.Centre()
        self.Show(True)

    def OnEncode(self, event):
        dlg = wx.TextEntryDialog(None, 'Enter your password:', 'Password')
        if dlg.ShowModal() == wx.ID_OK:
            password = dlg.GetValue()
            dlg.Destroy()
        else:
            return

        with wx.FileDialog(self, "Open Image file", wildcard="Image files (*.png)|*.png",
                          style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:
            if fileDialog.ShowModal() == wx.ID_CANCEL:
                return
            imgpath = fileDialog.GetPath()

        data = self.textCtrl.GetValue()
        if not data:
            wx.MessageBox('Data is empty', 'Info', wx.OK | wx.ICON_INFORMATION)
            return

        encrypted_data = xor_crypt(data, password)
        image = Image.open(imgpath, 'r')
        newimg = image.copy()
        encode_enc(newimg, encrypted_data)

        with wx.FileDialog(self, "Save Image file", wildcard="PNG files (*.png)|*.png",
                          style=wx.FD_SAVE | wx.FD_OVERWRITE_PROMPT) as fileDialog:
            if fileDialog.ShowModal() == wx.ID_CANCEL:
                return
            savepath = fileDialog.GetPath()
            newimg.save(savepath, "PNG")

        wx.MessageBox('Encoding complete!', 'Info', wx.OK | wx.ICON_INFORMATION)

    def OnDecode(self, event):
        dlg = wx.TextEntryDialog(None, 'Enter your password:', 'Password')
        if dlg.ShowModal() == wx.ID_OK:
            password = dlg.GetValue()
            dlg.Destroy()
        else:
            return

        with wx.FileDialog(self, "Open Image file", wildcard="Image files (*.png)|*.png",
                          style=wx.FD_OPEN | wx.FD_FILE_MUST_EXIST) as fileDialog:
            if fileDialog.ShowModal() == wx.ID_CANCEL:
                return
            imgpath = fileDialog.GetPath()

        image = Image.open(imgpath, 'r')
        data = ''
        imgdata = iter(image.getdata())

        while (True):
            pixels = [value for value in imgdata.__next__()[:3] +
                                    imgdata.__next__()[:3] +
                                    imgdata.__next__()[:3]]
            binstr = ''
            for i in pixels[:8]:
                if (i % 2 == 0):
                    binstr += '0'
                else:
                    binstr += '1'
            data += chr(int(binstr, 2))
            if (pixels[-1] % 2 != 0):
                break

        decrypted_data = xor_crypt(data, password)
        self.textCtrl.SetValue(decrypted_data)

def main():
    app = wx.App(False)
    frame = Steganography(None)
    app.MainLoop()

if __name__ == '__main__':
    main()